In [1]:
import pandas as pd
import pyomo.environ as pye
import pyomo.opt as pyo

![image info](https://miro.medium.com/max/1280/1*y4AHwh75uQ771dEdO6sxJg.png)

# Costs

In [2]:
# Inbound Costs
df_inbound = pd.read_csv(r"D:\Downloads\Chrome Downloads\supply-chain-optimization-main\new1\supply-planning-main\df_inprice.csv", index_col = 1)
df_inbound.drop(columns="Unnamed: 0", inplace = True)
df_inbound

,D1,D2
FROM,,
P1,3.0,5.0
P2,2.3,6.6


In [3]:
# Outbound Costs
df_outbound = pd.read_csv(r"D:\Downloads\Chrome Downloads\supply-chain-optimization-main\new1\supply-planning-main\df_outprice.csv", index_col = 1)
df_outbound.drop(columns="Unnamed: 0", inplace = True)
df_outbound

,S1,S2,S3,S4,S5,S6,S7,S8,S9,S10,...,S191,S192,S193,S194,S195,S196,S197,S198,S199,S200
from,,,,,,,,,,,,,,,,,,,,,
D1,2.30,4.23,2.26,3.38,1.59,2.01,5.32,6.63,2.38,6.62,...,5.86,8.30,3.02,1.01,2.77,2.96,3.53,8.60,2.77,7.06
D2,5.31,2.18,8.52,8.34,4.59,1.04,1.89,6.45,8.35,3.32,...,7.54,2.11,4.33,1.54,4.75,7.84,8.21,4.51,3.27,3.13


# capacity

In [4]:
# Production capacity
df_prod = pd.DataFrame({
    'plant': ['P1','P2'],
    'max': [300, 200]})
df_prod = df_prod.set_index('plant')
df_prod

,max
plant,
P1,300
P2,200


In [6]:
# Cross-Docking Capacity
df_t = pd.DataFrame({
    'DC': ['D1','D2'],
    'CAPACITY': [200, 200]})
df_t = df_t.set_index('DC')
df_t

,CAPACITY
DC,
D1,200
D2,200


# Demand

In [7]:
# Demand
df_demand = pd.read_csv(r"D:\Downloads\Chrome Downloads\supply-chain-optimization-main\new1\supply-planning-main\df_demand.csv", index_col = 1)
df_demand.drop(columns="Unnamed: 0", inplace = True)
df_demand

,DEMAND
STORE,
S1,244
S2,172
S3,124
S4,90
S5,158
...,...
S196,57
S197,52
S198,243


# Assigning Lists and Dict

In [8]:
store_list = list(df_demand.index)
plant_list = ["P1", "P2"]
distr_list = ["D1", "D2"]

In [9]:
outbound_dict = dict(df_outbound.stack())
outbound_dict

{('D1', 'S1'): 2.3,
 ('D1', 'S2'): 4.23,
 ('D1', 'S3'): 2.26,
 ('D1', 'S4'): 3.38,
 ('D1', 'S5'): 1.59,
 ('D1', 'S6'): 2.01,
 ('D1', 'S7'): 5.32,
 ('D1', 'S8'): 6.63,
 ('D1', 'S9'): 2.38,
 ('D1', 'S10'): 6.62,
 ('D1', 'S11'): 6.53,
 ('D1', 'S12'): 3.67,
 ('D1', 'S13'): 3.8,
 ('D1', 'S14'): 8.02,
 ('D1', 'S15'): 7.83,
 ('D1', 'S16'): 6.15,
 ('D1', 'S17'): 7.17,
 ('D1', 'S18'): 4.74,
 ('D1', 'S19'): 4.91,
 ('D1', 'S20'): 4.71,
 ('D1', 'S21'): 2.95,
 ('D1', 'S22'): 3.36,
 ('D1', 'S23'): 2.32,
 ('D1', 'S24'): 4.25,
 ('D1', 'S25'): 3.09,
 ('D1', 'S26'): 1.31,
 ('D1', 'S27'): 3.12,
 ('D1', 'S28'): 6.66,
 ('D1', 'S29'): 8.05,
 ('D1', 'S30'): 2.41,
 ('D1', 'S31'): 5.5,
 ('D1', 'S32'): 3.73,
 ('D1', 'S33'): 1.48,
 ('D1', 'S34'): 3.96,
 ('D1', 'S35'): 8.67,
 ('D1', 'S36'): 2.13,
 ('D1', 'S37'): 7.28,
 ('D1', 'S38'): 5.19,
 ('D1', 'S39'): 4.09,
 ('D1', 'S40'): 1.49,
 ('D1', 'S41'): 6.64,
 ('D1', 'S42'): 8.36,
 ('D1', 'S43'): 3.24,
 ('D1', 'S44'): 1.65,
 ('D1', 'S45'): 1.38,
 ('D1', 'S46'): 2.97,


In [10]:
inbound_dict = dict(df_inbound.stack())
inbound_dict

{('P1', 'D1'): 3.0, ('P1', 'D2'): 5.0, ('P2', 'D1'): 2.3, ('P2', 'D2'): 6.6}

In [11]:
prod_cap_dict = dict({i: df_prod.loc[i,"max"]*100 for i in plant_list})
prod_cap_dict

{'P1': 30000, 'P2': 20000}

In [12]:
distr_cap_dict = dict({i: df_t.loc[i,"CAPACITY"]*100 for i in distr_list})
distr_cap_dict

{'D1': 20000, 'D2': 20000}

In [13]:
demand_dict = dict({i:df_demand.loc[i,"DEMAND"] for i in store_list})
demand_dict

{'S1': 244,
 'S2': 172,
 'S3': 124,
 'S4': 90,
 'S5': 158,
 'S6': 175,
 'S7': 269,
 'S8': 223,
 'S9': 123,
 'S10': 129,
 'S11': 26,
 'S12': 286,
 'S13': 296,
 'S14': 292,
 'S15': 165,
 'S16': 265,
 'S17': 123,
 'S18': 246,
 'S19': 122,
 'S20': 118,
 'S21': 114,
 'S22': 259,
 'S23': 261,
 'S24': 79,
 'S25': 297,
 'S26': 150,
 'S27': 201,
 'S28': 284,
 'S29': 247,
 'S30': 289,
 'S31': 160,
 'S32': 185,
 'S33': 241,
 'S34': 218,
 'S35': 221,
 'S36': 61,
 'S37': 69,
 'S38': 193,
 'S39': 58,
 'S40': 80,
 'S41': 277,
 'S42': 136,
 'S43': 190,
 'S44': 187,
 'S45': 280,
 'S46': 18,
 'S47': 138,
 'S48': 259,
 'S49': 49,
 'S50': 16,
 'S51': 221,
 'S52': 156,
 'S53': 61,
 'S54': 252,
 'S55': 249,
 'S56': 97,
 'S57': 21,
 'S58': 89,
 'S59': 16,
 'S60': 191,
 'S61': 290,
 'S62': 269,
 'S63': 162,
 'S64': 243,
 'S65': 280,
 'S66': 248,
 'S67': 28,
 'S68': 281,
 'S69': 136,
 'S70': 116,
 'S71': 183,
 'S72': 259,
 'S73': 286,
 'S74': 29,
 'S75': 15,
 'S76': 112,
 'S77': 277,
 'S78': 103,
 'S79': 18,
 

# Model Creation

In [14]:
model = pye.ConcreteModel()

### Lists as Sets

In [15]:
model.stores = pye.Set(initialize = store_list)
model.plants = pye.Set(initialize = plant_list)
model.distributions = pye.Set(initialize = distr_list)

### Dict as Params

In [16]:
model.outbound_cost = pye.Param(model.distributions, model.stores, initialize = outbound_dict)
model.inbound_cost = pye.Param(model.plants, model.distributions, initialize = inbound_dict)
model.prod_cap = pye.Param(model.plants, initialize = prod_cap_dict)
model.distr_cap = pye.Param(model.distributions, initialize = distr_cap_dict)
model.store_demand = pye.Param(model.stores, initialize = demand_dict)

### Desicion making Variables as Variables

In [17]:
model.inbound_quantity = pye.Var(model.plants, model.distributions, domain = pye.NonNegativeReals)
model.outbound_quantity = pye.Var(model.distributions, model.stores, domain = pye.NonNegativeReals)

### Contraints

In [18]:
def con_demand(model, j):
    return sum(model.outbound_quantity[i,j] for i in model.distributions) >= model.store_demand[j]
model.con_demand = pye.Constraint(model.stores, rule=con_demand)

In [19]:
def con_prod_cap(model, i):
    return sum(model.inbound_quantity[i,j] for j in model.distributions) <= model.prod_cap[i]
model.con_prod_cap = pye.Constraint(model.plants, rule=con_prod_cap)

In [20]:
def con_distr_cap(model, j):
    return sum(model.inbound_quantity[i,j] for i in model.plants) <= model.distr_cap[j]
model.con_distr_cap = pye.Constraint(model.distributions, rule=con_distr_cap)

In [21]:
def con_in_out(model, j):
    return sum(model.inbound_quantity[i,j] for i in model.plants) == sum(model.outbound_quantity[j, k] for k in model.stores)
model.con_in_out = pye.Constraint(model.distributions, rule = con_in_out)

### Objective - Linear equation

![image info](https://miro.medium.com/max/906/1*sJ764LEC8fnn2b3ddgNqxw.gif)

In [22]:
expr = sum(model.inbound_cost[i,j]*model.inbound_quantity[i,j] for i in model.plants for j in model.distributions) + sum(model.outbound_cost[i,j]*model.outbound_quantity[i,j] for i in model.distributions for j in model.stores)
model.objective = pye.Objective(expr = expr)

### Selecting Solver and optimization

In [23]:
solver = pyo.SolverFactory("glpk")
results = solver.solve(model,tee=True)

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write C:\Users\rshib\AppData\Local\Temp\tmpgdg4nvua.glpk.raw --wglp C:\Users\rshib\AppData\Local\Temp\tmpybaol1vy.glpk.glp
 --cpxlp C:\Users\rshib\AppData\Local\Temp\tmp6so2gve0.pyomo.lp
Reading problem data from 'C:\Users\rshib\AppData\Local\Temp\tmp6so2gve0.pyomo.lp'...
207 rows, 405 columns, 813 non-zeros
2250 lines were read
Writing problem data to 'C:\Users\rshib\AppData\Local\Temp\tmpybaol1vy.glpk.glp'...
2037 lines were written
GLPK Simplex Optimizer 5.0
207 rows, 405 columns, 813 non-zeros
Preprocessing...
206 rows, 404 columns, 812 non-zeros
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 206
      0: obj =   0.000000000e+00 inf =   3.181e+04 (200)
    201: obj =   2.779652100e+05 inf =   0.000e+00 (0)
*   283: obj =   2.228907300e+05 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION F

# Optimal quantity to produce and supply to minimize cost

In [24]:
model.display()

Model unknown

  Variables:
    inbound_quantity : Size=4, Index=inbound_quantity_index
        Key          : Lower : Value   : Upper : Fixed : Stale : Domain
        ('P1', 'D1') :     0 :     0.0 :  None : False : False : NonNegativeReals
        ('P1', 'D2') :     0 : 11806.0 :  None : False : False : NonNegativeReals
        ('P2', 'D1') :     0 : 20000.0 :  None : False : False : NonNegativeReals
        ('P2', 'D2') :     0 :     0.0 :  None : False : False : NonNegativeReals
    outbound_quantity : Size=400, Index=outbound_quantity_index
        Key            : Lower : Value : Upper : Fixed : Stale : Domain
          ('D1', 'S1') :     0 : 244.0 :  None : False : False : NonNegativeReals
         ('D1', 'S10') :     0 :   0.0 :  None : False : False : NonNegativeReals
        ('D1', 'S100') :     0 :   0.0 :  None : False : False : NonNegativeReals
        ('D1', 'S101') :     0 : 270.0 :  None : False : False : NonNegativeReals
        ('D1', 'S102') :     0 :   0.0 :  None :

### No. of objectives, constraints & variables

In [25]:
results.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 222890.73
  Upper bound: 222890.73
  Number of objectives: 1
  Number of constraints: 207
  Number of variables: 405
  Number of nonzeros: 813
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.05064702033996582
# ----------------------------------------------------------
#   Solution Information
# -----------------------